In [ ]:
# before proceeding, you should have already ran notebook 1.1 so that there is an asari results directory to work with

import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# lets find the previous results and load the composite map

asari_subdir = None
for x in os.listdir("../../"):
    if x.startswith("Results_MT02_results"):
        asari_subdir = os.path.join(os.path.abspath("../../"), x)

cmap = pickle.load(open(os.path.join(asari_subdir, 'export', 'cmap.pickle'), 'rb'))

In [ ]:
# confirming that the cmap is of type dict and examining the keys it contains.

print(type(cmap))
print(cmap.keys())



In [ ]:
# the composite map contains multiple mass tracks. These can be accessed as follows:

cmap['list_mass_tracks'][583]

In [ ]:
# lets define a function for plotting mass tracks

def plotk(track, color='m', start=100, end=400, yticks=[0, 5e7, 1e8], title=''):
    plt.figure(figsize=(8, 4))
    X = range(start, min(len(track['intensity']), end))
    Y = track['intensity'][start: min(len(track['intensity']), end)]
    plt.plot(X, Y, color=color, marker='o', markersize=4, linestyle='dashed')
    if title:
        plt.title(title)


In [ ]:
# and lets use it to plot some tracks.

plotk(cmap['list_mass_tracks'][3], color='k', start=00, end=400, yticks=[0, 2e8])


In [ ]:
# note that the composite mass track is from cumulated signals from all samples. How do we look at the same mass track in individual samples?
# Asari stores the mapping of composite tracks to individual sample tracks in the _mass_grid_mapping file.

cmap_to_sample_map = pd.read_csv(os.path.join(asari_subdir, 'export', '_mass_grid_mapping.csv'), index_col=0)
cmap_to_sample_map


In [ ]:
myfiles = ['batch11_MT_20210805_005', 'batch12_MT_20210806_005_20210806121327', 'batch13_MT_20210807_005']
cmap_to_sample_map.iloc[3, :]

In [ ]:
# the above output shows the mapping of the cmap mass track to mass tracks in the samples. They should be integers but are not due to pandas inferring them as float. Next we can use this mapping to load the per sample mass tracks as follows: 

data = []
for f in myfiles:
    with open(os.path.join(asari_subdir, 'pickle', f+'.pickle'), 'rb') as fh:
        data.append(pickle.load(fh))

In [ ]:
# lets take a look at the sample mass tracks
data[0].keys()

In [ ]:
# lets do some plotting, lets plot the same mass track for all the samples in myfiles

for f, d in zip(myfiles, data):
    x = cmap_to_sample_map.iloc[3][f]
    plotk(d['list_mass_tracks'][int(x)], title=f)

In [ ]:
# with the above data plotted, now we can explain composte mass tracks a different way. 
# composite mass tracks are produced by aligning then summing the sample mass tracks. here we skip the alignment but perform the summation. 

composite = {
    'intensity': None
}

for f, d in zip(myfiles, data):
    x = cmap_to_sample_map.iloc[3][f]
    lmt = d['list_mass_tracks'][int(x)]['intensity'][:400]
    if composite['intensity'] is None:
        composite['intensity'] = lmt
    else:
        composite['intensity'] += lmt



In [ ]:
plotk(composite)

# this composite is very similar to the mass track we plotted earlier from the cmap file. 
# in asari, the peak picking of this composite yields features, and the corresponding regions in each sample are then referenced to yield the intensity per sample. 